# Pandas的SettingWithCopyWarning的报警

## 0.读取数据

In [1]:
import pandas as pd

In [2]:
fpath = 'beijing_last_12_weather.csv'
df = pd.read_csv(fpath)


In [3]:
df.head()

,日期,最高温,最低温,天气,风力风向
0,2024-12-01 星期日,15℃,-1℃,多云,西北风 2级
1,2024-12-02 星期一,4℃,-4℃,晴,西北风 3级
2,2024-12-03 星期二,9℃,-3℃,晴~多云,西北风 2级
3,2024-12-04 星期三,8℃,-2℃,晴,西北风 2级
4,2024-12-05 星期四,9℃,-2℃,晴~阴,西北风 3级


In [6]:
df.loc[:,"最高温"] = df["最高温"].str.replace('℃','').astype('int32')
df.loc[:,"最低温"] = df["最低温"].str.replace('℃','').astype('int32')

AttributeError: Can only use .str accessor with string values!

In [10]:
df.head()

,日期,最高温,最低温,天气,风力风向
0,2024-12-01 星期日,15,-1,多云,西北风 2级
1,2024-12-02 星期一,4,-4,晴,西北风 3级
2,2024-12-03 星期二,9,-3,晴~多云,西北风 2级
3,2024-12-04 星期三,8,-2,晴,西北风 2级
4,2024-12-05 星期四,9,-2,晴~阴,西北风 3级


## 1.复现

In [13]:
# 只选出3月份的数据用于分析
condition = df['日期'].str.startswith("2024-12")

In [14]:
# 设置温差
df[condition]["wencha"] = df['最高温']-df['最低温']

D:\Temp\ipykernel_11496\2999045345.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[condition]["wencha"] = df['最高温']-df['最低温']


In [16]:
# 查看是否创建成功
#当发出这个SettingWithCopyWarning: 时有时会修改成功,有时不会
df[condition].head()

,日期,最高温,最低温,天气,风力风向
0,2024-12-01 星期日,15,-1,多云,西北风 2级
1,2024-12-02 星期一,4,-4,晴,西北风 3级
2,2024-12-03 星期二,9,-3,晴~多云,西北风 2级
3,2024-12-04 星期三,8,-2,晴,西北风 2级
4,2024-12-05 星期四,9,-2,晴~阴,西北风 3级


## 2、原因
- 发出警告的代码df[condition]["wencha"] = df['最高温']-df['最低温']
- 相当于: df.get(condition).set(wen_cha)，第一步骤的get发出了报警
- 链式操作其实是两个步骤，先get后set，get得到的dataframe可能是view也可能是copy，pandas发出警告
- 核心要诀:pandas的dataframe的修改写操作，只允许在源dataframe上进行，一步到位

## 3.解决方法1:
- 将get+set的两步操作,改成set的一步操作

In [20]:
df.loc[condition,'wencha'] = df["最高温"]- df['最低温']


In [21]:
df[condition].head()

,日期,最高温,最低温,天气,风力风向,wencha
0,2024-12-01 星期日,15,-1,多云,西北风 2级,16
1,2024-12-02 星期一,4,-4,晴,西北风 3级,8
2,2024-12-03 星期二,9,-3,晴~多云,西北风 2级,12
3,2024-12-04 星期三,8,-2,晴,西北风 2级,10
4,2024-12-05 星期四,9,-2,晴~阴,西北风 3级,11


## 4.解决方法2:
-如果需要预筛选数据做后续的处理分析,使用copy复制dataframe

In [24]:
df_month12 = df[condition].copy()

In [25]:
df_month12.head()

,日期,最高温,最低温,天气,风力风向,wencha
0,2024-12-01 星期日,15,-1,多云,西北风 2级,16
1,2024-12-02 星期一,4,-4,晴,西北风 3级,8
2,2024-12-03 星期二,9,-3,晴~多云,西北风 2级,12
3,2024-12-04 星期三,8,-2,晴,西北风 2级,10
4,2024-12-05 星期四,9,-2,晴~阴,西北风 3级,11


In [26]:
df_month12["wencha"] = df["最高温"]- df['最低温']

In [27]:
df_month12.head()


,日期,最高温,最低温,天气,风力风向,wencha
0,2024-12-01 星期日,15,-1,多云,西北风 2级,16
1,2024-12-02 星期一,4,-4,晴,西北风 3级,8
2,2024-12-03 星期二,9,-3,晴~多云,西北风 2级,12
3,2024-12-04 星期三,8,-2,晴,西北风 2级,10
4,2024-12-05 星期四,9,-2,晴~阴,西北风 3级,11


#### 总之，pandas不允许先筛选子dataframe，再进行修改写入
#### 要么使用.1oc实现一个步骤直接修改源dataframe要么先复制一个子dataframe再一个步骤执行修改 